# HFCLKAUDIO LQR frequency controller model and simulation

In [ ]:
%matplotlib widget
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt

plt.close("all")
# We use overflow intentionally
np.seterr(over="ignore");

Model where input is applied incrementally, and changes in frequency take effect instantly. Characterization of the PLL (see `hfclkaudio_dynamics.ipynb`) shows that this is a good approximation because the dynamics damp out in <10us.

In [ ]:
dt = 0.1
# Scale the gain by 2^32 because the frequency ratio is scaled likewise
k_u = 2**32 * 32e6 / (12 * 2**16 * 11289600)
A = np.array([[1, dt], [0, 1]])
B = np.array([[dt * k_u], [k_u]])
C = np.array([[1, 0]])
D = np.array([[0]])
sys = scipy.signal.StateSpace(A, B, C, D, dt=dt)
display(sys)

u_step = 10
t = np.arange(0, 1, dt)
u = np.zeros_like(t)
u[2] = u_step
t, y, x = scipy.signal.dlsim(sys, u)

fig, ax = plt.subplots()
ax.plot(t, y)
display(k_u)

Compute the LQR feedback for the system

In [ ]:
# From: https://github.com/modestyachts/robust-control-from-vision/blob/be9ae48f78cbf6fa2a12d13f0df6784e1ede9039/lqr.py#L5-L11
def optimal_k(A, B, R, P):
    return scipy.linalg.inv(B.T.dot(P).dot(B) + R).dot(B.T.dot(P).dot(A))


def lqr_inf_horizon(A, B, Q, R):
    P = scipy.linalg.solve_discrete_are(A, B, Q, R)
    K = optimal_k(A, B, R, P)
    return K, P


Q = np.diag([1.0, 1e-3])
R = np.array([[5.0]])

K, _ = lqr_inf_horizon(sys.A, sys.B, Q, R)
display(K)

Simulation of controller

In [ ]:
PHASE_FRAC = np.uint64(1 << 32)


def qu32_32_from_int(ticks: np.uint32) -> np.uint64:
    return np.uint64(ticks) << np.uint8(32)


def phase_add_float(theta: np.uint64, inc: np.float32) -> np.uint64:
    if inc >= 0:
        theta += np.uint64(inc)
    else:
        theta -= np.uint64(-inc)
    return np.uint64(theta)


def phase_diff_signed(a: np.uint64, b: np.uint64) -> np.int64:
    return np.int64(np.uint64(a - b))


def freq_ctlr(t, nominal_freq, k_u, k_theta, k_f, max_step, theta_0, f_0):
    last_time = np.uint64(0)

    t *= nominal_freq
    u = np.empty(t.shape[0], dtype=np.int16)
    theta = np.empty(t.shape[0], dtype=np.uint64)
    f = np.empty(t.shape[0], dtype=np.float32)

    target_theta = np.uint64(0)

    for i, local_time in enumerate(t):
        if i == 0:
            u[i] = 0
            theta[i] = theta_0
            f[i] = f_0 * PHASE_FRAC
            last_time = local_time
            continue

        dt = (local_time - last_time).astype(np.float32) / PHASE_FRAC
        last_time = local_time

        u_scaled = u[i - 1].astype(np.float32) * k_u
        theta[i] = phase_add_float(theta[i - 1], dt * (f[i - 1] + u_scaled))
        f[i] = f[i - 1] + u_scaled

        theta_err = phase_diff_signed(target_theta, theta[i]).astype(np.float32)
        f_err = -f[i]
        u_unclipped = np.round(k_theta * theta_err + k_f * f_err)
        u[i] = np.clip(u_unclipped, -max_step, max_step).astype(np.uint16)

    # Convert internal representations to meaningful units
    t = t.astype(np.float64) / nominal_freq
    theta = phase_diff_signed(theta, 0).astype(np.float64) / PHASE_FRAC
    f /= PHASE_FRAC

    return t, theta, f, u

In [ ]:
nominal_freq = np.float32(16e6)
q_theta = np.float32(0)
q_f = np.float32(256)
r = np.float32(390625)
k_theta = K[0, 0]
k_f = K[0, 1]
max_step = 1000
theta_0 = 0
f_0 = 1e-5

t = np.arange(0, 100, dt, dtype=np.float64)
t, theta, f, u = freq_ctlr(
    t,
    nominal_freq=nominal_freq,
    k_u=k_u,
    k_theta=k_theta,
    k_f=k_f,
    max_step=max_step,
    theta_0=theta_0,
    f_0=f_0,
)

fig, (ax_freq, ax_theta, ax_u) = plt.subplots(3, 1, figsize=(6, 8), sharex=True)
ax_freq.plot(t, f)
ax_freq.set_title("Frequency")
ax_freq.set_ylabel("f")

ax_theta.plot(t, theta)
ax_theta.set_title("Phase")
ax_theta.set_ylabel("θ")
ax_theta.set_xlabel("t")

ax_u.plot(t, u)
ax_u.set_title("Input")
ax_u.set_xlabel("t");